In [ ]:
!curl -s -OL https://raw.githubusercontent.com/StephenZou-bot/sd-webui-notebook/main/a1111/base.py
!curl -s -OL https://raw.githubusercontent.com/StephenZou-bot/sd-webui-notebook/main/a1111/get_ip.py
!curl -s -OL https://raw.githubusercontent.com/StephenZou-bot/sd-webui-notebook/main/a1111/pastebin.py
%run base.py

## Frp config

In [ ]:
config_content = """
# frpc.toml
serverAddr = "x.x.x.x"
serverPort = 7000

[[proxies]]
name = "ssh"
type = "tcp"
localIP = "127.0.0.1"
localPort = 22
remotePort = 6000
"""

with open('frpc.toml', 'w') as file:
    file.write(config_content.strip())


### Download from Pastebin URL

In [ ]:
pastebin_url = "https://pastebin.com/BYafVgUS" # example link
hf_token = "" # if use private repo
civitai_api_key = ""
#=====================================================
%run pastebin.py --pastebin_url {pastebin_url}  --hf_token {hf_token} --civitai_api_key {civitai_api_key}


## Link Datasets

In [ ]:
import os

def link_kaggle_files(input_dir, output_dir):
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):

        relative_path = os.path.relpath(root, input_dir)

        target_root = os.path.join(output_dir, relative_path)
        if not os.path.exists(target_root):
            os.makedirs(target_root)

        for file in files:
            source_file = os.path.join(root, file)
            target_file = os.path.join(target_root, file)
            if not os.path.exists(target_file):
                os.symlink(source_file, target_file)
            else:
                print(f"Skipping existing file: {target_file}")

datasets = {
    "/kaggle/input/checkpoints": f"{ui}/stable-diffusion-webui/models/Stable-diffusion/",
    "/kaggle/input/checkpoint": f"{ui}/stable-diffusion-webui/models/ControlNet/",
    "/kaggle/input/load-vae": f"{ui}/stable-diffusion-webui/models/VAE/"
}

for input_dir, output_dir in datasets.items():
    link_kaggle_files(input_dir, output_dir)
    print(f"Linked files from {input_dir} to {output_dir}")


## Run WebUI

In [ ]:
ngrok_token = "" # https://dashboard.ngrok.com/get-started/your-authtoken
zrok_token = "" # optional, good ngrok alternative
password = "sdw2024"
#================================================
%run /kaggle/working/get_ip.py

tunnel_port= 7860
tunnel = Tunnel(tunnel_port)

tunnel.add_tunnel(command="cl tunnel --url localhost:{port}", name="cl", pattern=re.compile(r"[\w-]+\.trycloudflare\.com"))
tunnel.add_tunnel(command="lt --port {port}", name="lt", pattern=re.compile(r"[\w-]+\.loca\.lt"), note="Password : " + Fore.GREEN + public_ipv4 + Style.RESET_ALL + " rerun cell if 404 error.")
if zrok_token:
    !zrok enable {zrok_token} &> /dev/null
    tunnel.add_tunnel(command="zrok share public http://localhost:{port}/ --headless", name="zrok", pattern=re.compile(r"[\w-]+\.share\.zrok\.io"))

with tunnel:
    %cd {ui}/stable-diffusion-webui
    !echo -n {start_colab} >{ui}/stable-diffusion-webui/static/colabTimer.txt
#     !venv/bin/python launch.py --port=7860 --api --ngrok {ngrok_token} --xformers --theme dark --enable-insecure-extension-access --disable-console-progressbars --disable-safe-unpickle --no-half-vae
    !venv/bin/python launch.py --port=7860 --api --ngrok {ngrok_token} --encrypt-pass={password} --xformers --theme dark --enable-insecure-extension-access --disable-console-progressbars --disable-safe-unpickle --no-half-vae